# Quantum networks in the quantum cloud
Suppose, we have different quantum processors (with different parameters) connected through a classical network, then what is the most efficient way to provide link between  any two quantum processors?

In [33]:
import networkx as nx 
import math
G = nx.Graph()
G.add_edge(1,2)
G.add_edge(1,3)
G.add_edge(2,3)
G.add_edge(2,4)
G.add_edge(3,5)
G.add_edge(4,5)
G.add_edge(4,6)
G.add_edge(5,6)
#nx.draw(G, with_labels=True)
num_edges = nx.number_of_edges(G)
param_matrix = [[0]*num_edges]*num_edges
#define with [distance (km), coupling eff. (%), num_qubits, attenuation length (km)]
param_matrix[1][2] = [30, 0.95, 10, 20]
param_matrix[1][3] = [30, 0.5, 20, 20]
param_matrix[2][3] = [30, 0.2, 30, 20]
param_matrix[2][4] = [30, 0.69, 8, 20]
param_matrix[3][5] = [30, 0.34, 4, 20]
param_matrix[4][5] = [30, 0.67, 5, 20]
param_matrix[4][6] = [30, 0.8, 20, 20]
param_matrix[5][6] = [30, 0.2, 10, 20]
#We could print the list of all paths between two nodes (1,6) in the network using the simple function
node_a = 1
node_b = 6
paths = list(nx.all_simple_paths(G, node_a, node_b, cutoff=None))

# Entanglement generation (nodes)
We could start with entangled link generation between two nodes. 
Strategy: Try to create entangled pairs between "m" qubits. in case, none of them suceed, try again (try "neg" times). 
Probability of getting one pair (prob) = (1-(1-p)**(neg*m))
time taken = neg*dist/c (km/s in fiber)

In [34]:
def elink_edges(G, i,j, param):
    dist = param[i][j][0]
    latt = param[i][j][3]
    pc = (param[i][j][1])#coupling efficiency
    p = (pc**2)* math.exp(-dist/latt)
    num_pairs = math.ceil(param[i][j][2]/len(list(G.edges(i)))) #number of pairs (edges) for one elink
    rate = 0
    for neg in range(1,10):
        prob = (1-(1-p)**(neg*num_pairs)) #probability
        time = neg*dist/(2*1e5)
        if prob/time > rate:
            rate = prob*(1/time)
            qubits = num_pairs
            prob_max = prob
            time_comm = time
    return rate, qubits, prob_max, time_comm
def rate(links, elink):
    t_max = 0
    prob = 1
    qubits = 0
    for var in range(len(links)-1):
        prob= prob*elink[links[var]][links[var+1]][2]
        if elink[links[var]][links[var+1]][3]> t_max:
            time =  elink[links[var]][links[var+1]][3]
        qubits = qubits + elink[links[var]][links[var+1]][1]
    return qubits/(prob/time), qubits, time


# Optimization with cost function

The optimized path is obtained by calculating by defining a cost function and find the path that yields the minimum cost

In [35]:
cost = [0]*(len(paths))
qubits_tot = [0]*(len(paths))
time = [0]*(len(paths))
elink = [[0]*num_edges]*num_edges
for a in range(len(paths)):
    for b in range(len(paths[a])-1):
        node_1 = paths[a][b]
        node_2 = paths[a][b+1]
        elink[node_1][node_2]= elink_edges(G, node_1,node_2, param_matrix)
for i in range(len(paths)):
    cost[i], qubits_tot[i], time[i] = rate(paths[i], elink)
print("The opimum path to connect nodes", node_a, "and", node_b, "based on the cost function is" ,paths[cost.index(min(cost))])
print("The total number of qubits utilized" , qubits_tot[cost.index(min(cost))])
print("The total time for Bell-pair generation" , time[cost.index(min(cost))], "seconds")

The opimum path to connect nodes 1 and 6 based on the cost function is [1, 2, 4, 6]
The total number of qubits utilized 11
The total time for Bell-pair generation 0.00015 seconds
